<a href="https://colab.research.google.com/github/kabilan942/Hugging-Face/blob/main/Hugging_Face_Summarization_in_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Exploring Dataset**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
     |████████████████████████████████| 120 kB 45.6 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 51.1 MB/s 
     |████████████████████████████████| 115 kB 50.8 MB/s 
     |████████████████████████████████| 127 kB 67.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!git config --global user.email "kabilanp942@gmail.com"
!git config --global user.name "Kabilan.P"

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

english_dataset = load_dataset("amazon_reviews_multi", "en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

In [5]:
english_dataset['train'].features 

{'review_id': Value(dtype='string', id=None),
 'product_id': Value(dtype='string', id=None),
 'reviewer_id': Value(dtype='string', id=None),
 'stars': Value(dtype='int32', id=None),
 'review_body': Value(dtype='string', id=None),
 'review_title': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'product_category': Value(dtype='string', id=None)}

In [6]:
english_dataset['train'][5]

{'review_id': 'en_0420650',
 'product_id': 'product_en_0705898',
 'reviewer_id': 'reviewer_en_0155342',
 'stars': 1,
 'review_body': 'Poor quality. The material was fuzzy from day one. It got discolored in less than a month, even though we kept it outdoors under a covered porch. I returned this item.',
 'review_title': 'Poor quality. The material was fuzzy from day one',
 'language': 'en',
 'product_category': 'industrial_supplies'}

Since each review is accompanied by a short title, we can use the titles as the target summaries for our model to learn from.

Therefore, the review information we are interested in is contained in the **review_body** and review_title columns.

We'll look at a few examples by creating a simple function that takes a random sample from the training set.

We can create a random sample by chaining the **Dataset.shuffle()** and **Dataset.select()** functions together.

The **seed** (= 42 here) is fixed in Dataset.shuffle() for reproducibility purposes.

Dataset.select() expects an iterable of indices, so we’ve passed **range(num_samples)** to grab the first 3 examples from the shuffled dataset.


In [7]:
def show_samples(dataset, num_samples=3, seed=42):
  sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))   
  # dataset['train'] used as Dataset for shuffle() and dataset["train"].shuffle(seed=seed) is used as Dataset for select()
  for example in sample:
    print()
    print(">> Title: {}".format(example['review_title']))
    print(">> Review: {}".format(example['review_body']))

show_samples(english_dataset)   # num_samples = 3 and seed = 42


>> Title: Worked in front position, not rear
>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.

>> Title: meh
>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue

>> Title: Can't beat these for the money
>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. Some have r

We'll focus on generating summaries for a single domain of products, since training the model on all 400,000 reviews (en+es) will take too long.

To choose the domain, convert english_dataset to Pandas.DataFrame

In [8]:
english_dataset.set_format('pandas')

In [9]:
english_dataset['train'][:].head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,en_0964290,product_en_0740675,reviewer_en_0342986,1,Arrived broken. Manufacturer defect. Two of th...,I'll spend twice the amount of time boxing up ...,en,furniture
1,en_0690095,product_en_0440378,reviewer_en_0133349,1,the cabinet dot were all detached from backing...,Not use able,en,home_improvement
2,en_0311558,product_en_0399702,reviewer_en_0152034,1,I received my first order of this product and ...,The product is junk.,en,home
3,en_0044972,product_en_0444063,reviewer_en_0656967,1,This product is a piece of shit. Do not buy. D...,Fucking waste of money,en,wireless
4,en_0784379,product_en_0139353,reviewer_en_0757638,1,went through 3 in one day doesn't fit correct ...,bubble,en,pc


In [10]:
english_df = english_dataset['train'][:]
english_df.shape

(200000, 8)

In [11]:
english_df['product_category'].value_counts().shape

(31,)

Let's print the **top 20** products from the dataframe.

In [12]:
english_df['product_category'].value_counts()[:20]

home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: product_category, dtype: int64

Switching the format of english_dataset from "pandas" back to "arrow":

In [13]:
english_dataset.reset_format()

Let's focus on summarizing book reviews; the categories that comes under book review are **book** and **digital_ebook_purchase**

In [14]:
def filter_books(example):   # example -> dataset which was called in the call 
  return (example['product_category']=='book' or example['product_category']=='digital_ebook_purchase' )
    
# filter_books is the function which will be passed as the function in the filter()

In [15]:
english_books = english_dataset.filter(filter_books)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

Sanity Check:

In [16]:
show_samples(english_books)


>> Title: I'm dissapointed.
>> Review: I guess I had higher expectations for this book from the reviews. I really thought I'd at least like it. The plot idea was great. I loved Ash but, it just didnt go anywhere. Most of the book was about their radio show and talking to callers. I wanted the author to dig deeper so we could really get to know the characters. All we know about Grace is that she is attractive looking, Latino and is kind of a brat. I'm dissapointed.

>> Title: Good art, good price, poor design
>> Review: I had gotten the DC Vintage calendar the past two years, but it was on backorder forever this year and I saw they had shrunk the dimensions for no good reason. This one has good art choices but the design has the fold going through the picture, so it's less aesthetically pleasing, especially if you want to keep a picture to hang. For the price, a good calendar

>> Title: Helpful
>> Review: Nearly all the tips useful and. I consider myself an intermediate to advanced use

In [17]:
english_books

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 10505
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 231
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 278
    })
})

Notice english_books has only **10505** samples from the books domain in the training set out of the 200,000 samples. 

We’ll **filter out the examples with very short titles** so that our model can produce more interesting summaries. Since we’re dealing with English and Spanish texts, we can use a rough heuristic to split the titles on whitespace and then use Dataset.filter() method

In [18]:
english_books = english_books.filter(lambda x: (len(x['review_title'].split())>2))

# the above filter filters out the samples from the dataset which has less than 2 words in its title (words seperated by a white space)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
english_books

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 6162
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 135
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 164
    })
})

Notice once filtered, that the number of samples is down to **6162** from **10505** in the training set of books_dataset.

## **Preprocessing the data**

In [20]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Example of tokenization using 't5-small'

In [21]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [27, 1858, 1183, 8, 26049, 5880, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

['▁I', '▁loved', '▁reading', '▁the', '▁Hunger', '▁Games', '!', '</s>']

The special Unicode character ▁ and end-of-sequence token </s> indicate that we’re dealing with the SentencePiece tokenizer, which is based on the **Unigram segmentation algorithm**.

It is especially useful for multilingual corpora since it allows SentencePiece to be agnostic about accents, punctuation, and the fact that many languages, like Japanese, do not have whitespace characters.

We apply truncation to both the reviews and their titles to ensure we don’t pass excessively long inputs to our model.

The tokenizer has a **as_target_tokenizer()** function that allows you to tokenize the labels in parallel to the inputs.

This is typically done using a context manager inside a preprocessing function that first encodes the inputs, and then encodes the labels as a separate column.

In [23]:
# Values for max_input_length and max_target_length, which set the upper limits for how long our reviews and titles can be
# Since the review body is typically much larger than the title, the values are scaled accordingly

max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['review_body'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (Title)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['review_title'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('review_title') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs



In [24]:
tokenized_dataset = english_books.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

We use batched=True in our Dataset.map() function above. This encodes the examples in batches of 1,000 (the default) and allows to make use of the multithreading capabilities of the fast tokenizers in Transformers.

## **Creating a Baseline**

A common baseline for text summarization is to simply take the first three sentences of an article, often called the lead-3 baseline. We could use full stops to track the sentence boundaries, but this will fail on acronyms like “U.S.”

Se we'll use the nltk library

In [25]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download the punctuation rules:

In [26]:
import nltk 

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
from nltk.tokenize import sent_tokenize

# s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''
# sent_tokenize(s)
# ['Good muffins cost $3.88\nin New York.', 'Please buy me\ntwo of them.', 'Thanks.']

# join method
# myTuple = ("John", "Peter", "Vicky")
# x = "#".join(myTuple); print(x)
# OP: John#Peter#Vicky

def three_sentence_summary(text):   # function to extract 1st 3 sentences in the text
  return '\n'.join(sent_tokenize(text)[:3])   # first 3 sentence only

print(three_sentence_summary(english_books['train'][3]['review_body']))


Very cheap looking.
Thin pages and messy illustrations.
Definitely not what I would choose to read to my class.


Let's define a function which extracts these 'summaries' (1st 3 sentences) and compute rouge scores for them with 'summaries' as predictions and the 'review_title' as references in .compute() method

In [28]:
def evaluate_baseline(dataset, metric):
  summaries = [three_sentence_summary(text) for text in dataset['review_body']]
  return metric.compute(predictions=summaries, references=dataset['review_title'])

We can use this function to compute rouge scores over the validation set.

In [29]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8601d97905d8d52fc28612d5526899dba1b3183cc08f675d34c22a6e93e86fce
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [30]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.9 MB/s 
     |████████████████████████████████| 1.3 MB 10.9 MB/s 


In [31]:
import evaluate

rouge_score = evaluate.load('rouge')

In [32]:
import pandas as pd

# x = dict(name = "John", age = 36, country = "Norway")
# OP: {'name': 'John', 'age': 36, 'country': 'Norway'}

# round() -> returns a floating point number that is a rounded version of the specified number, with the specified number of decimals.
# round(5.76543, 2) -> 5.77

score = evaluate_baseline(english_books['validation'], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn]*100, 2)) for rn in rouge_names)

rouge_dict

{'rouge1': 15.22, 'rouge2': 8.58, 'rougeL': 14.36, 'rougeLsum': 14.75}

We can see that the rouge2 score is significantly lower than the rest; this likely reflects the fact that review titles are typically concise and so the lead-3 baseline is too verbose (using or expressed in more words than are needed).

## **Fine-tuning T5 with the Trainer API**

In [33]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)  # model_checkpoint = "t5-small"

We’ll need to generate summaries in order to compute ROUGE scores during training.

Hugging Face transformers provides Seq2SeqTrainingArguments and Seq2SeqTrainer classes that can do this automatically.

In [34]:
print(len(tokenized_dataset["train"]))
print(len(tokenized_dataset["train"])//8)

6162
770


In [35]:
from transformers import Seq2SeqTrainingArguments

# parameters of Seq2SeqTrainingArguments
batch_size = 8
num_training_epochs = 8
logging_steps = len(tokenized_dataset["train"])//8

args = Seq2SeqTrainingArguments(evaluation_strategy='epoch',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-finetuned-amazon-english",
                                push_to_hub=True)

# predict_with_generate argument has been set to True to indicate that we should generate summaries
# during evaluation so that we can compute ROUGE scores for each epoch
# The decoder performs inference by predicting tokens one by one, and this is implemented by the model’s 
# generate() method. Setting predict_with_generate=True tells the Seq2SeqTrainer to use that method for evaluation.
# save_total_limit = 3 -> because even the “small” version of T5 uses around a GB of hard drive space

For summarization evaluation is a bit more involved than simply calling rouge_score.compute() on the model’s predictions, since we need to decode the outputs and labels into text before we can compute the ROUGE scores.

In [36]:
import numpy as np

def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

Next, we need to define a data collator for our sequence-to-sequence task. Since T5 is an encoder-decoder Transformer model, one subtlety with preparing our batches is that during decoding we need to shift the labels to the right by one. This is required to ensure that the decoder only sees the previous ground truth labels and not the current or future ones, which would be easy for the model to memorize.

Hugging Face Transformers provides a **DataCollatorForSeq2Seq collator** that will dynamically pad the inputs and the labels for us. 

In [37]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

First, we need to remove the columns with strings because the collator won’t know how to pad these elements

In [38]:
len(english_books['train'].column_names)

8

In [39]:
len(tokenized_dataset['train'].column_names)

11

In [40]:
tokenized_dataset = tokenized_dataset.remove_columns(english_books['train'].column_names)

In [41]:
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

Collator expects a list of dicts, where each dict represents a single example in the dataset

In [42]:
features = [tokenized_dataset['train'][i] for i in range(2)]
data_collator(features)

# Notice we have info ['input_ids', 'attention_mask', 'labels'] for 2 examples

{'input_ids': tensor([[ 3059,  4779,  6833,    65,   349,    38,     8,   167, 19930,   120,
           147, 25444,  4346,     5,  1203,    27,  6220,     8,  2291,    19,
             3,     9, 24556,     5, 20510,  9412,     5,     1],
        [   27,  5563,    48,   484,    30,  2083,  7806,    94,   470,  4363,
             5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]]), 'labels': tensor([[  499,  8581,    48,   484,  3290,    22,    17,   414,     1],
        [25629,    12,  1299,   484,     1,  -100,  -100,  -100,  -100]]), 'decoder_input_ids': tensor([[    0,   499,  8581,    48,   484,  3290,    22,    17,   414],
        [    0, 25629,    12,  1299,   484,     1,     0, 

The first example is longer than the second one, so the input_ids and attention_mask of the second example have been padded on the right with a [PAD] token (whose ID is 0). Similarly, we can see that the labels have been padded with -100s, to make sure the padding tokens are ignored by the loss function. And finally, we can see a new decoder_input_ids which has shifted the labels to the right by inserting a [PAD] token in the first entry.

In [43]:
from transformers import Seq2SeqTrainer

# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["validation"], 
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric)

In [44]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6162
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6168


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.358300,3.251274,16.619100,9.003200,16.065300,15.897300
2,3.102200,3.214652,16.684200,9.377300,16.274500,16.159700
3,3.006700,3.171759,16.758000,8.755100,16.348800,16.258300
4,2.948200,3.185434,18.817900,9.488700,18.365400,18.160000
5,2.893400,3.166862,18.799900,9.964700,18.169700,17.939400
6,2.838900,3.178216,18.534000,9.263800,17.932400,17.704900
7,2.817400,3.170916,18.356500,9.686700,17.731300,17.538600
8,2.800000,3.171259,19.105400,9.888600,18.407700,18.138900


Saving model checkpoint to t5-finetuned-amazon-english/checkpoint-500
Configuration saved in t5-finetuned-amazon-english/checkpoint-500/config.json
Model weights saved in t5-finetuned-amazon-english/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-finetuned-amazon-english/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-finetuned-amazon-english/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 135
  Batch size = 8
Saving model checkpoint to t5-finetuned-amazon-english/checkpoint-1000
Configuration saved in t5-finetuned-amazon-english/checkpoint-1000/config.json
Model weights saved in t5-finetuned-amazon-english/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-finetuned-amazon-english/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-finetuned-amazon-english/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5-finetuned-amazon-english/checkpoint-1500
Configu

TrainOutput(global_step=6168, training_loss=2.9704758771507964, metrics={'train_runtime': 735.6486, 'train_samples_per_second': 67.01, 'train_steps_per_second': 8.384, 'total_flos': 1945884680847360.0, 'train_loss': 2.9704758771507964, 'epoch': 8.0})

Once the training is complete, we can see the final ROUGE scores by running trainer.evaluate():

In [ ]:
trainer.evaluate()

## **Using the Fine-Tuned Model**

Pre-Trained model -> "t5-small"

Pushing the model weights (after fine-tuning) to the hub

In [ ]:
trainer.push_to_hub(commit_message='Training_complete', tags='Summarization')

This will save the checkpoint and configuration files to output_dir, before uploading all the files to the Hub. By specifying the tags argument, we also ensure that the widget on the Hub will be one for a summarization pipeline instead of the default text generation one associated with the T5 architecture

The output from trainer.push_to_hub() is a URL to the Git commit hash

In [ ]:
from transformers import pipeline

hub_model = 't5-finetuned-amazon-english'
summarizer = pipeline('classification', model=hub_model)

print_summary() function to show the review, title, and generated summary together

In [ ]:
def print_summary(idx):
  review = english_books['test'][idx]['review_body']
  title = english_books['test'][idx]['review_title']
  summarizer_result = summarizer(english_books['test'][idx]['review_body'])
  summary = summarizer(english_books['test'][idx]['review_body'])[0]['summary_text']
  print(summarizer_result)
  print(">>> Review: {}".format(review))
  print(">>> Title: {}".format(title))
  print(">>> Summary: {}".format(summary))

In [ ]:
print_summary(5)

In [ ]:
print_summary(50)